In [93]:
import re
import pandas as pd
from pathlib import Path
import openpyxl
from openpyxl import Workbook
import pycountry
import pycountry
import json

#read
data_folder = Path("C:/Users/Jing Li/Desktop/cv2db/project/Inputs")
file = data_folder / "Carly Roster_JL.xlsx"

# wb = openpyxl.load_workbook(file)
# ws = wb["CONSULTANTS"]

# id = ws["A2":"A123"]
# countries = ws["G2":"G123"]

In [94]:
#open consultant sheet
Carly_data = pd.ExcelFile(file)
df = Carly_data.parse("CONSULTANTS")

#Countries worked/visited/studied
cnty = []
for i in df["COUNTRIES WORKED/VISITED/STUDIED"]:
    i = str(i)
    temp = []
    for country in pycountry.countries:
        if re.search(country.name, i):
            temp.append(country.name)
    cnty.append(temp)

In [95]:
#Countries missing
cnty[16].append("Tanzania")
cnty[18].remove("Dominica")
cnty[18].remove("Niger")
cnty[21].append('United States')

df["Countries worked/studied"] = cnty

In [96]:
#Languages
lang_dic = ['Amharic', 'Arabic', 'Bemba', 'Bengali', 'Chinese', 'Creole', 'Danish', 'English',
            'French', 'German', 'Hindi','Kinyarwanda', 'Kirundi', 'Kiswahili', 'Luo', 'Malayalam',
            'Nyanja', 'Portuguese','Russian','Setswana', 'Spanish', 'Swahili', 'Swedish',
            'Tagalog', 'Tigrigna', 'Tigrinya', 'Yoruba']

lang = []

for i in df["FOREIGN LANGUAGE(S)"]:
    i = str(i)
    temp = [j for j in lang_dic if re.search(j, i)]
    lang.append(temp)
    
df["Languages"] = lang

In [97]:
# Extract Expertises
expertises = {'Coaching & Training': ["coach"],
              'Federal government agency': ["federal"],
              'Meeting Facilitation': ["meeting"],
              'Report Writing': ["report", "proposal", "grant"],
              'Budget and fiscal management': ["budget", "fiscal manage",
                                               "fiscal consul", "fiscal oversight"],
              'Epidemiology': ['Epidemiology'],
              'Global health': ["global health"],
              'Healthcare policy and regulations': ["policy", "regulation"],
              'Infectious Diseases': ["Infectious"],
              'International Relations': ["international relation"],
              'Mental Health': ['mental health'],
              'Psychiatry': ["Psychiatry"],
              'Substance abuse': ["substance abuse"],
              'Human Trafficking': ["trafficking"],
              'Capacity building': ["capacity building", "capacity develop",
                                    "building their capacity", "build their capacity",
                                   "build capacity", "services capacity", "system capacity",
                                   "capacity expansion", "built capacity"],
              'Clinic Operations': ["clinic"],
              'Financial analysis & management': ["financial", "finance", "accounting"],
              'Fund development & Sustainability': ["fund ", "funds", "funding"],
              'Grants management': ["grants"],
              'Human Capital Systems': ["Human Capital"],
              'Human resources management': ["Human resource"],
              'Humility and Organizational Leaders': ['Humility'],
              'Organizational Development&Transformation': ['Organizational Development',
                                                           'Transformation'],
              'Quality improvement and management': ["quality program", "quality improve",
                                                     "improve quality"],
              'State Procurement System': ["Procurement"],
              'Strategic planning': ["Strategic"],
              'Data collection, analysis and reporting': ["data collect", "data analy", 'analysis'],
              'Family Planning and Reproductive Health': ["family planning", "FP/RH", "reproductive"],
              'Program evaluation & analysis': ["evaluation", "evalute", "evaluator"],
              'Programmatic Assessment & Management': ["Programmatic"]
             }

def Check(p_data, dic):
    p = []
    for key in dic:
        for kw in dic[key]:
            if re.search(kw, p_data, re.IGNORECASE):
                p.append(key)
                break
    return p

Expertises = [Check(str(p), expertises) for p in df['EXPERTISE']]

# Store
df["Expertises"] = Expertises

In [98]:
# Special population

population = {'Early Childhood': ["early child"],
              'Elderly population': ["elder", "senior health"],
              'Female population': ["female", "girls", "women"],
              'HIV/AIDS population': ["hiv", "aids"],
              'Homeless population': ["homeless"],
              'Infants': ["infants", "baby"],
              'LGBTQ': ["lgbt", "transgender"],
              'Low-income population': ["low-income", "poor", "poverty"],
              'Maternal and Children': ["mother", "maternal", "child"],
              'People with disability': ["disable", "disabilit"],
              'Rural Health': ["rural"],
              'Youth': ["youth", "adolescent"],
              'Tribal Home Visiting': ["tribal"],
             }
Population = [Check(str(p), population) for p in df['EXPERTISE']]

# Store
df["Target Population"] = Population

In [108]:
# Federal project

# Special population

project = {'Global Trade Analysis Project (GTAP)': ['GTAP'],
           'Health Centers (BPHC)   ': ['BPHC'],
           'HIV/AIDS Domestic (HAB Ryan White)': ['Ryan White'],
           'HIV/AIDS International (HAB PEPFAR)': ['PEPFAR'],
           'Maternal and Child Heath (MCHB, ACF)': ['MCHB'],
           'Maternal, Infant, and Early Childhood Home Visiting Program (MIECHV)': ['MIECHV'],
           'Population Assessment of Tobacco and Health Study (PATH)': ['PATH'],
           'Rural Health (FORHP)': ['FORHP'],
          }
Project = [Check(str(p), project) for p in df['EXPERTISE']]

# Store
df["Federal Pro"] = Project

In [109]:
wb = Workbook()
interested_cols = ["Countries worked/studied", "Languages", "Expertises",
                   "Federal Pro", "Target Population"]

def normalization(col_name):
    output = []
    for i in range(len(df)):
        for j in df[col_name][i]:
            output.append([df['id'][i], j])
    return output

In [110]:
for i in interested_cols:
    ws = wb.create_sheet(title=i[:4])
    result = normalization(i)
    row = 1
    for j in result: #each j is a [people, skill] list
        ws["A"+str(row)] = j[0]
        ws["B"+str(row)] = i
        ws["C"+str(row)] = j[1]
        row += 1

In [111]:
# store
wb.save(data_folder / "Carly atomized.xlsx")